<a href="https://colab.research.google.com/github/zglx09/project-based-learning/blob/main/python_api_w_pagination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective:** \
An API pull that handles pagination, to get details of all projects associated with an Initiative within Linear.

**Additional Information:** \
Within Linear, A project management tool:

*   The Project's module sits under Initiatives.
*   An Initiative can have multiple Projects, and a Project can belong to multiple Initiative.
*   Linear's API uses GraphQL.

In [ ]:
import requests

# API endpoint and headers
url = "https://api.linear.app/graphql"
headers = {
    "Authorization": "{linear_api_key}",
    "Content-Type": "application/json"
}

def fetch_all_initiatives():
    all_initiatives_to_proj = []
    page_cursor = None

    while True:
        # Construct the GraphQL query
        query = {
            "query": """
            query ($first: Int, $after: String) {
                initiativeToProjects(first: $first, after: $after) {
                    nodes {
                        id
                        initiative { id name status description createdAt targetDate creator { id }}
                        project { id name }
                    }
                    pageInfo {
                        hasNextPage
                        endCursor
                    }
                }
            }
            """,
            "variables": {
                "first": 30,
                "after": page_cursor
            }
        }

        # Make the POST request to the API
        response = requests.post(url, headers=headers, json=query)

        if response.status_code == 200:
            data = response.json()

            # Extract initiatives and projects data
            initiatives_to_proj = data.get('data', {}).get('initiativeToProjects', {}).get('nodes', [])
            all_initiatives_to_proj.extend(initiatives_to_proj)

            # Handle pagination
            page_info = data.get('data', {}).get('initiativeToProjects', {}).get('pageInfo', {})
            page_cursor = page_info.get('endCursor')

            # Stop if there are no more pages
            if not page_info.get('hasNextPage'):
                break
        else:
            # Handle the error case
            return {'error': f'Request failed with status code {response.status_code}', 'response': response.text}

    # Return the fetched initiatives and projects
    return {'initiatives_projs': all_initiatives_to_proj}

# Call the function and print the results
result = fetch_all_initiatives()
print(result)